# Modéliser le problème de l'EDT sous forme de programme linéaire

## 1. Les données de base

### Les créneaux

In [ ]:
%pip install sympy
%pip install requests
%pip install pulp

In [ ]:
possible_start_times = [480, 570, 665, 855, 945, 1040] # en minutes depuis minuit... 480 = 8h00

days = ["m", "tu", "w", "th", "f", 'sa', 'su'] # monday, tuesday, wednesday, thursday, friday, saturday, sunday

class Slot:
    def __init__(self, day:str, start_time:int, duration:int) -> None:
        self.day = day
        self.start_time = start_time
        self.duration = duration

    @property
    def end_time(self) -> int:
        return self.start_time + self.duration

    def __repr__(self) -> str:
        return f"{self.day} {self.start_time//60}h{self.start_time%60:02}"

    def __le__(self, other) -> bool:
        return days.index(self.day) < days.index(other.day) or (self.day==other.day and self.start_time <= other.start_time)

### Les groupes

In [ ]:

from typing import Self
class StructuralGroup:
    def __init__(self, name:str, is_basic:bool, train_prog: str, parent_group):
        self.name = name
        self.train_prog=train_prog
        self.parent_group = parent_group
        self.is_basic = is_basic

    def __repr__(self) -> str:
        return f"{self.train_prog}-{self.name}"

    def and_ancestors(self) -> set[Self]:
        if self.parent_group is None:
            return {self}
        else:
            return {self}|self.parent_group.and_ancestors()

### Les profs

In [ ]:
class Tutor:
    def __init__(self, username:str) -> None:
        self.username = username

    def __repr__(self) -> str:
        return self.username

class UserPreference(Slot):
    def __init__(self, day: str, start_time:int, duration:int, user:Tutor, value:int) -> None:
        super().__init__(day, start_time, duration)
        self.user = user
        self.value = value # 0 = unavailable, 1-8 = preference degree

    def __repr__(self) -> str:
        return super().__str__() + f" : {self.value} ({self.user})"

### Les cours

In [ ]:
class Module:
    def __init__(self, abbrev:str) -> None:
        self.abbrev = abbrev

    def __repr__(self) -> str:
        return self.abbrev

class Course:
    def __init__(self, course_type:str, tutor:Tutor, group:StructuralGroup, module:Module, room_type:str) -> None:
        self.type = course_type
        self.tutor = tutor
        self.group = group
        self.module = module
        self.room_type = room_type

    def __repr__(self) -> str:
        return f"{self.type}-{self.module} gr{self.group.name} ({self.tutor.username})"

### Fabriquons maintenant les objets utiles à notre problème

In [ ]:
groups={}
for train_prog in "BUT1","BUT2","BUT3":
    groups[train_prog] =  {"CE": StructuralGroup("CE",  # BUT2.CE
                                                 is_basic=False,
                                                 train_prog=train_prog,
                                                 parent_group=None)}
    groups[train_prog]['12'] = StructuralGroup(name='12',  # BUT2.12
                                               is_basic=False,
                                               train_prog=train_prog,
                                               parent_group = groups[train_prog]['CE'])

    for i in range(1,5):
        if i in [1,2]:
            parent = groups[train_prog]['12']
        else:
            parent = groups[train_prog]['CE']
        groups[train_prog][str(i)] = StructuralGroup(name=str(i), # BUT2.1, BUT2.2, BUT2.3, BUT2.4
                                                     is_basic=False,
                                                     train_prog=train_prog,
                                                     parent_group = parent)
    for i in range(1,5):
        for lettre in ["A", "B"]:
            groups[train_prog][str(i)+lettre] = StructuralGroup(name=str(i)+lettre,
                                                                is_basic=True,
                                                                train_prog=train_prog,
                                                                parent_group = groups[train_prog][str(i)]) # BUT2.1A, BUT2.1B, BUT2.2A, etc...

slots = [Slot(day, start_time, 90)
         for day in days
         for start_time in possible_start_times]



On va également récupérer des informations directement depuis une api : une liste de cours et les disponibilités des profs

In [ ]:
import requests

url_courses = "https://flopedt.iut-blagnac.fr/fr/api/courses/courses/?dept=INFO&week=6&year=2026"
r = requests.get(url_courses)
courses_dict_list = r.json()

# url_availabilities = "https://flopedt.iut-blagnac.fr/fr/api/preferences/user-default/?dept=INFO"
# r = requests.get(url_availabilities)
# availabilities_dict_list = r.json()

Nous pouvons maintenant fabriquer des listes d'objets de la classe Course, et récupérer au passage les profs et les modules (dans un dictionnaire dont les clefs seront les accronymes), ainsi qu'une liste d'objets de la classe UserPreference

In [ ]:
tutors = {}
courses = []
modules = {}

for course in courses_dict_list:
    # On ignore les cours qui ne sont pas des cours de BUT2
    training_program = course['groups'][0]['train_prog']
    if training_program!='BUT2':
        continue
    group=groups[training_program][course['groups'][0]['name']]
    tutor_username=course['tutor']
    if tutor_username not in tutors:
        tutors[tutor_username] = Tutor(tutor_username)
    module_abbrev = course['module']['abbrev']
    if module_abbrev not in modules:
        modules[module_abbrev] = Module(module_abbrev)
    courses.append(Course(course_type=course['type']['name'],
                          tutor=tutors[tutor_username],
                          group=group,
                          module=modules[module_abbrev],
                          room_type=course['room_type']['name']
                          )
                  )

# availabilities=[]
# for availability in availabilities_dict_list:
#     if availability['user'] in tutors:
#         availabilities.append(UserPreference(day=availability['day'],
#                                              start_time=availability['start_time'],
#                                              duration=availability['duration'],
#                                              user=tutors[availability['user']],
#                                              value=availability['value'])
#                              )

print("Les profs sont :", tutors)
print("Les modules :", modules)

## 2. Les variables du programme linéaire
On va construire, pour *chaque* cours et *chaque* créneau, une variable **binaire** qui signifie "ce cours aura lieu sur ce créneau". On va stocker ces variables dans un dictionnaire scheduled :

In [ ]:
# %pip install pulp
from pulp import LpProblem, LpVariable, LpMinimize, LpMaximize, LpStatus, value, lpSum, LpBinary, LpAffineExpression

scheduled={}
for slot in slots:
    for course in courses:
        scheduled[course, slot] = LpVariable(f"x_{course}_{slot}_{i}", cat=LpBinary)

On crée ensuite le problème flop, et la fonction coût qu'on ajoutera au problème à la fin:

In [ ]:
flop = LpProblem("flop", LpMinimize)
cost = LpAffineExpression()

Essayons maintenant d'écrire quelques contraintes ou préférences d'emploi du temps.

Commençons par le commencement :  chaque cours doit être affecté à un créneau (et un seul!)

In [ ]:
i=0
for course in courses:

    maSomme=0
    for slot in slots:
        maSomme += scheduled[course, slot]

    flop += maSomme == 1, f"Le cours {course} est placé {i}"
    i+=1

print(flop)

Une autre : sur chaque créneau, chaque prof a au plus un cours.

In [ ]:
for slot in slots:
    for tutor == PSE
        

In [ ]:
flop.solve

Faisons quelque chose de similaire pour les groupes (de 'BUT2'), en commençant par l'un d'eux : le groupe "3A" a au plus un cours par créneau.

Attention : lorsque l'on parle du groupe '3A', on parle aussi de ses *surgroupes* --> *ancestors*

Et maintenant des contraintes plus spécifiques :

In [ ]:
# Pas de cours le jeudi après-midi

In [ ]:
#Pour chaque module, le CM doit être avant les TD


In [ ]:
# Minimiser l'utilisation du premier et du dernier créneau de chaque jour (cout de 2)
for slot in slots:
    if slot.start_time == ... :
        cost += ...

# Minimiser l'utilisation de l'avant dernier créneau de chaque jour (cout de 1)
for slot in slots:
    ...

Enfin, on ajoute la fonction objectif au problème, et on le résoud.

In [ ]:
#On ajoute le coût à la fonction objectif, à minimiser
flop += cost, "Coût"

#On lance la résolution
flop.solve()

Voyons le statut, la valeur de chaque variable, 

In [ ]:
# Impression du statut
print("Statut:", LpStatus[flop.status])

# Impression de chaque variable avec sa valeur dans la solution optimale
for (slot, course) in scheduled:
    var = scheduled[slot, course]
    if var.varValue == 1:
        print(f"{slot} | {course}")

# Impression de l'objectif optimal
print("Coût total = ", value(flop.objective))